I've spent a lot of time looking at the course notes and I've decided to deviate a little from the previous structure of the course here. The reason is that we can't really talk about ggplot without having some basic mastery of manipulating tabular data.

A brief philosophical note